## Make Tim2DDriver work
- For `observe!` from `Tim2DDriver` to work, had to make the following changes
    - Had to modify `AutomotiveDrivingModels.jl/src/2d/vehicles/scene_records.jl` to extend
the default length of the capacity in the record container from 100 to 300
    - Did the same capacity extension in `Records.jl/src/frames.jl`, `QueueRecords.jl` and 
`ListRecords.jl`
- Note that querying Tim2DDriver yields a `latLonAccel` return type. Had to create an array
to enable these values to work with `step`

## Changes to environment file
- `LatLonAccel` instead of `AccelTurnRate` because ego vehicles are
driven using Tim2DDriver that outputs lat lon accel
- Changed the render method to not save images in `/tmp` but instead
return the render frame to be able to make videos using Reel

In [ ]:
using NGSIM
using AutomotiveDrivingModels
using AutoViz
using Interact # Make video in notebook
using Reel # Save video as gif
using CSV # For writing to csv
using DataFrames # For writing to csv
using PyPlot # For in notebook plotting

In [ ]:
using AutoEnvs

n_veh = 10 # Number of ego vehicles
# filepath = joinpath(dirname(pathof(NGSIM)), "..", 
#     "data", "trajdata_i80_trajectories-0400-0415.txt")
filepath = joinpath(dirname(pathof(NGSIM)), "..", 
    "data", "trajdata_i101_trajectories-0750am-0805am.txt")
params = Dict(
        "trajectory_filepaths"=>[filepath],
        "H"=>200,
        "primesteps"=>50,
        "n_veh"=>n_veh,
        "remove_ngsim_veh"=>false
)
# env = MultiagentNGSIMEnvVideoMaker(params)
env = MultiagentNGSIMEnv(params);
timestep = 0.1;

In [ ]:
# Sanity check: See if what we have loaded makes sense by rendering a scene
# Test like this is what helped catch the NGSIM occursin order reversing issue
randi = 5
reset(env,random_seed=randi)
render(env.scene,env.roadway)

In [ ]:
model = Tim2DDriver(timestep,mlane = MOBIL(timestep));

In [ ]:
# Borrowed from `sisl/gail-driver/validation/validation.jl`
# Reduced T to 0.1 to see more aggressive driving
# Now wondering how to make the blue vehicle change lanes
mlon = IntelligentDriverModel(v_des = 20.0, σ=2.5,k_spd=1.0,T=0.5,s_min=2.0,a_max=3.0,d_cmf=2.5)
mlat = ProportionalLaneTracker(σ=0.1, kp=3.0, kd=2.0)
mlane = MOBIL(timestep,politeness=0.0,advantage_threshold=0.0,safe_decel=3.0)
#mlane = TimLaneChanger(timestep)
model = Tim2DDriver(timestep, mlon=mlon, mlat=mlat, mlane=mlane);

In [ ]:
# TODO: How to define a function with only kwargs and no args?
function run_one_sim(numsteps)
    @show numsteps
    traj = zeros(numsteps,n_veh,66) # 66 observations
    
    data_rmse_t = zeros(numsteps,2)
    data_rmse_vel = zeros(numsteps,2)
    data_rmse_pos = zeros(numsteps,2)
    
    hard_brake_sum = 0
    
    for ii in 1:numsteps
    
        a = zeros(env.n_veh,2)
        
        for (jj,veh) in enumerate(env.ego_vehs)
            observe!(model,env.scene,env.roadway,veh.id)
            latlonacc = rand(model)
            a[jj,1] = latlonacc.a_lat
            a[jj,2] = latlonacc.a_lon
        end
        
        x, r, terminal, info = step(env, a)
        
#         @show info # info contains the rmse information
        
        # Extract rmse information from simulation
        vec_rmse_t = info["rmse_t"]
        vec_rmse_pos = info["rmse_pos"]
        vec_rmse_vel = info["rmse_vel"]
        
        # Compute means over all the agents in this timestep
        mean_rmse_t = mean(vec_rmse_t)
        mean_rmse_vel = mean(vec_rmse_vel)
        mean_rmse_pos = mean(vec_rmse_pos)
        
        # Store rmse into array where column 1 is time and col 2 is the value
        data_rmse_t[ii,1] = ii-1; data_rmse_t[ii,2] = mean_rmse_t
        data_rmse_vel[ii,1] = ii-1; data_rmse_vel[ii,2] = mean_rmse_vel
        data_rmse_pos[ii,1] = ii-1; data_rmse_pos[ii,2] = mean_rmse_pos
        
        
        
        traj[ii,:,:] = x # x is the features: its num_ego_veh x 66
        
#         @show sum(x[:,19]) # 19th column of x captures the is colliding 1 or 0
        @show size(x)
        @show "yo"
        accel_vec = x[:,9]
        
        hard_brake_sum += sum(accel_vec .< -1.0)
        
        dleft_vec = x[:,21]; @show dleft_vec
        dright_vec = x[:,22]; @show dright_vec
        @show min(dleft_vec,dright_vec)
        
        @show find(min(x[:,21],x[:,22])<=-1.0)
    end
    
    # Plotting the rmse stuff
#     figure(0)
#     plot(data_rmse_pos[:,2])
#     figure(1)
#     plot(data_rmse_t[:,2])
#     figure(2)
#     plot(data_rmse_vel[:,2])
    return traj
end

In [ ]:
randi = 15
reset(env,random_seed=randi)
traj = run_one_sim(1);

In [ ]:
data_rmse_pos

In [ ]:
# If you wanted to write data to CSV files
CSV.write("rmse_t.csv",  DataFrame(data_rmse_t), writeheader=false)
CSV.write("rmse_vel.csv",  DataFrame(data_rmse_vel), writeheader=false)
CSV.write("rmse_pos.csv",  DataFrame(data_rmse_pos), writeheader=false)

In [ ]:
using PyPlot

In [ ]:
plot(data_rmse_pos)

## Video making and saving to file

In [ ]:
# Inspired from the drawsim method in AutoViz/doc/AutoViz.ipynb
function makevid(t,dt=NaN)
    
        a = zeros(env.n_veh,2)

        for (jj,veh) in enumerate(env.ego_vehs)
            observe!(model,env.scene,env.roadway,veh.id)

            latlonacc = rand(model)
            a[jj,1] = latlonacc.a_lat
            a[jj,2] = latlonacc.a_lon
        end
        x, r, terminal, _ = step(env, a)
    render(env)
end

In [ ]:
randi = 5
reset(env,random_seed=randi)
film = roll(makevid, fps=10, duration=5.0)
write("numveh_$(env.n_veh)_seed_$(randi)_timlane.gif",film)

## Video making using recorded cars on ngsim
- Trajdatas is what the car trajectories are stored in
- We want to color the ego vehicle differently to be able to see it

In [ ]:
td1 = load_trajdata(1)

scene = Scene(500)
temp_scene = get!(scene,td1,1000)
render(temp_scene,ROADWAY_101)

In [ ]:
# argument 1 loads i101 7:50 to 8:05.
# load_trajdata function defined in NGSIM.jl/src/trajdata.jl
td1 = load_trajdata(1); 

scene = Scene(500)
egoid = 546

# Drive here in the notebook. Replay the trajectory as recorded in the ngsim data
@manipulate for i in 2000:2000
    temp_scene = get!(scene,td1,i)
    
    carcolors = Dict{Int,Colorant}()
    for veh in temp_scene
        #@show veh.id
        # if veh id matches the egoid color it blue otherwise green
        carcolors[veh.id] = 
        in(veh.id, egoid) ? colorant"blue" : colorant"green"
    end
    render(temp_scene, ROADWAY_101, 
        cam=CarFollowCamera{Int}(546,5.0),
#         cam=StaticCamera(VecE2(1966400, 570900), 5.0),
#         cam=FitToContentCamera(0.),
        car_colors=carcolors)
end